In [76]:
from pathlib2 import Path
import os
import numpy as np
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torch.utils.data as data_utils
import torchvision
from torchvision import transforms
from torchvision import datasets
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset

In [39]:
ranges=[151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173
    , 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196,
       197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,
       220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242,
       243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265,
       266, 267, 268,281, 282, 283, 284, 285,32, 30, 31,33, 34, 35, 36, 37,80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90,
       91, 92, 93, 94, 95, 96, 97, 98, 99, 100,365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379,
       380, 381, 382,389, 390, 391, 392, 393, 394, 395, 396, 397,120, 121, 118, 119,300, 301, 302, 303, 304, 305, 306,
       307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319]


len(ranges)*50

10150

In [86]:
len(ranges)

203

In [40]:
synset_path = "/home/DATA/ITWM/ImageNet/ILSVRC2012_devkit_t12/data/synset_words.txt"
dir_class_map = {}
class_dir_map = {}
id_class_map  = {}

final = []

with open(synset_path, "rb") as map_class_file:
    rows = map_class_file.readlines()
    for idx, row in enumerate(rows):
        row = row.strip()
        arr = row.decode("utf-8").split(" ")
        # print(idx, arr[0], arr[1].split(",")[0])
        
        final.append((idx, arr[0], arr[1].split(",")[0]))
        
        # class_dir_map[arr[0]] = arr[2]
        # id_class_map[int(arr[1])] = arr[2]
        # dir_class_map[arr[0]] = int(arr[1])

     

In [35]:
final_arr = np.array(final)

In [41]:
selected_elements = [final[index] for index in ranges]

In [43]:
# for it in selected_elements:
#     print(it[1])

In [66]:
class RestrictedImageNet(torchvision.datasets.ImageFolder):
    def __init__(self, img_path, transform=None):
        super(RestrictedImageNet, self).__init__(img_path, transform)
        
        self._read_labels()
        # self.samples = self._make_dataset(self.samples)
        # self.classes, self.class_to_idx = self._my_classes()
        # self.imgs = self.samples
        # self.targets = [s[1] for s in self.samples]

    def _read_labels(self):
        map_clsloc_path = "/home/DATA/ITWM/ImageNet/ILSVRC2012_devkit_t12/data/map_clsloc.txt"
        dir_class_map = {}
        class_dir_map = {}
        id_class_map  = {}

        with open(map_clsloc_path, "rb") as map_class_file:
            rows = map_class_file.readlines()
            for row in rows:
                row = row.strip()
                arr = row.decode("utf-8").split(" ")
                class_dir_map[arr[0]] = arr[2]
                id_class_map[int(arr[1])] = arr[2]
                dir_class_map[arr[0]] = int(arr[1])
        
#         self.class_dir_map = class_dir_map
#         self.id_class_map  = id_class_map
        
#         with open('/home/DATA/ITWM/ImageNet/ILSVRC2012_devkit_t12/data/imagenet1000_clsidx_to_labels.txt') as f:
#             data = f.read()
#         import ast
#         d = ast.literal_eval(data)
        
#         key_list = list(d.keys())
#         val_list = list(d.values())
        
#         for key, value in class_dir_class_id.items():
#             for val in val_list:
#                 val_split = val.split(", ")
#                 val_split[0] = val_split[0].replace(" ", "_")
#                 if value in val_split:
#                     print( key,  key_list[ val_list.index(val) ], val )

#                     class_dir_class_id[key] = key_list[ val_list.index(val) ]


        self.dir_class_map = dir_class_map
        
        
    def _my_classes(self):
        self.dir_class_map 
        class_to_idx = {classes[i]: i for i in range(len(classes))}

        return classes, class_to_idx

    
    def _make_dataset(self, samples):
        n = len(samples)
        ds = [None] * n
        for i, (img, cls) in enumerate(samples):            
            ds[i] = (img, self._custom_class(cls))

        return ds
    
    
    # def _custom_class(self, cls):
    #     return self.dir_class_map[self.classes[cls]]

    
    def pil_loader(self, path):
        # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')
    

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        """
        path, target = self.samples[index]
        sample = self.pil_loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target

In [67]:
transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor()
])

In [68]:
dataset_root = '/home/DATA/ITWM/ImageNet'
val_root = os.path.join(dataset_root, 'val')

In [69]:
dataset = RestrictedImageNet(val_root, transform)

In [85]:
dataset.class_to_idx

{'n01440764': 0,
 'n01443537': 1,
 'n01484850': 2,
 'n01491361': 3,
 'n01494475': 4,
 'n01496331': 5,
 'n01498041': 6,
 'n01514668': 7,
 'n01514859': 8,
 'n01518878': 9,
 'n01530575': 10,
 'n01531178': 11,
 'n01532829': 12,
 'n01534433': 13,
 'n01537544': 14,
 'n01558993': 15,
 'n01560419': 16,
 'n01580077': 17,
 'n01582220': 18,
 'n01592084': 19,
 'n01601694': 20,
 'n01608432': 21,
 'n01614925': 22,
 'n01616318': 23,
 'n01622779': 24,
 'n01629819': 25,
 'n01630670': 26,
 'n01631663': 27,
 'n01632458': 28,
 'n01632777': 29,
 'n01641577': 30,
 'n01644373': 31,
 'n01644900': 32,
 'n01664065': 33,
 'n01665541': 34,
 'n01667114': 35,
 'n01667778': 36,
 'n01669191': 37,
 'n01675722': 38,
 'n01677366': 39,
 'n01682714': 40,
 'n01685808': 41,
 'n01687978': 42,
 'n01688243': 43,
 'n01689811': 44,
 'n01692333': 45,
 'n01693334': 46,
 'n01694178': 47,
 'n01695060': 48,
 'n01697457': 49,
 'n01698640': 50,
 'n01704323': 51,
 'n01728572': 52,
 'n01728920': 53,
 'n01729322': 54,
 'n01729977': 55,
 '

In [75]:
dataset_folder = ImageFolder(val_root, transform)

In [82]:
dataset_folder.imgs[0][1]

0

In [84]:
dataset_folder.class_to_idx

{'n01440764': 0,
 'n01443537': 1,
 'n01484850': 2,
 'n01491361': 3,
 'n01494475': 4,
 'n01496331': 5,
 'n01498041': 6,
 'n01514668': 7,
 'n01514859': 8,
 'n01518878': 9,
 'n01530575': 10,
 'n01531178': 11,
 'n01532829': 12,
 'n01534433': 13,
 'n01537544': 14,
 'n01558993': 15,
 'n01560419': 16,
 'n01580077': 17,
 'n01582220': 18,
 'n01592084': 19,
 'n01601694': 20,
 'n01608432': 21,
 'n01614925': 22,
 'n01616318': 23,
 'n01622779': 24,
 'n01629819': 25,
 'n01630670': 26,
 'n01631663': 27,
 'n01632458': 28,
 'n01632777': 29,
 'n01641577': 30,
 'n01644373': 31,
 'n01644900': 32,
 'n01664065': 33,
 'n01665541': 34,
 'n01667114': 35,
 'n01667778': 36,
 'n01669191': 37,
 'n01675722': 38,
 'n01677366': 39,
 'n01682714': 40,
 'n01685808': 41,
 'n01687978': 42,
 'n01688243': 43,
 'n01689811': 44,
 'n01692333': 45,
 'n01693334': 46,
 'n01694178': 47,
 'n01695060': 48,
 'n01697457': 49,
 'n01698640': 50,
 'n01704323': 51,
 'n01728572': 52,
 'n01728920': 53,
 'n01729322': 54,
 'n01729977': 55,
 '

In [70]:
dataset.classes

['n01440764',
 'n01443537',
 'n01484850',
 'n01491361',
 'n01494475',
 'n01496331',
 'n01498041',
 'n01514668',
 'n01514859',
 'n01518878',
 'n01530575',
 'n01531178',
 'n01532829',
 'n01534433',
 'n01537544',
 'n01558993',
 'n01560419',
 'n01580077',
 'n01582220',
 'n01592084',
 'n01601694',
 'n01608432',
 'n01614925',
 'n01616318',
 'n01622779',
 'n01629819',
 'n01630670',
 'n01631663',
 'n01632458',
 'n01632777',
 'n01641577',
 'n01644373',
 'n01644900',
 'n01664065',
 'n01665541',
 'n01667114',
 'n01667778',
 'n01669191',
 'n01675722',
 'n01677366',
 'n01682714',
 'n01685808',
 'n01687978',
 'n01688243',
 'n01689811',
 'n01692333',
 'n01693334',
 'n01694178',
 'n01695060',
 'n01697457',
 'n01698640',
 'n01704323',
 'n01728572',
 'n01728920',
 'n01729322',
 'n01729977',
 'n01734418',
 'n01735189',
 'n01737021',
 'n01739381',
 'n01740131',
 'n01742172',
 'n01744401',
 'n01748264',
 'n01749939',
 'n01751748',
 'n01753488',
 'n01755581',
 'n01756291',
 'n01768244',
 'n01770081',
 'n017